## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
import matplotlib as plt

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

In [7]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 64
What is the maximum temperature you would like for your trip? 95


In [8]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
5,5,Bredasdorp,ZA,-34.5322,20.0403,65.52,68,100,11.50,light rain
6,6,Georgetown,MY,5.4112,100.3354,79.90,85,20,3.00,few clouds
8,8,Mabaruma,GY,8.2000,-59.7833,84.45,62,100,6.35,overcast clouds
9,9,Victoria,HK,22.2855,114.1577,66.25,65,98,4.00,overcast clouds
10,10,Tuskegee,US,32.4240,-85.6916,73.22,54,68,5.75,broken clouds
11,11,Marawi,PH,7.9986,124.2928,65.64,98,89,3.96,overcast clouds
12,12,Ponta Do Sol,PT,32.6667,-17.1000,65.03,61,98,9.71,overcast clouds
15,15,Anloga,GH,5.7947,0.8973,84.42,77,47,10.42,scattered clouds
18,18,Marsh Harbour,BS,26.5412,-77.0636,75.70,64,0,10.49,clear sky
19,19,Calama,CL,-22.4667,-68.9333,79.29,10,99,23.02,overcast clouds


In [9]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                7
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [10]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
5,5,Bredasdorp,ZA,-34.5322,20.0403,65.52,68,100,11.50,light rain
6,6,Georgetown,MY,5.4112,100.3354,79.90,85,20,3.00,few clouds
8,8,Mabaruma,GY,8.2000,-59.7833,84.45,62,100,6.35,overcast clouds
9,9,Victoria,HK,22.2855,114.1577,66.25,65,98,4.00,overcast clouds
10,10,Tuskegee,US,32.4240,-85.6916,73.22,54,68,5.75,broken clouds
...,...,...,...,...,...,...,...,...,...,...
2021,2021,Bhigvan,IN,18.3167,74.7667,72.03,72,10,4.16,clear sky
2022,2022,Inyonga,TZ,-6.7167,32.0667,74.25,49,100,3.13,overcast clouds
2023,2023,Mayor Pablo Lagerenza,PY,-19.9309,-60.7718,93.85,47,100,5.08,light rain
2024,2024,Turiacu,BR,-1.6633,-45.3717,83.61,70,59,6.98,broken clouds


In [11]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Bredasdorp,ZA,65.52,light rain,-34.5322,20.0403,
6,Georgetown,MY,79.90,few clouds,5.4112,100.3354,
8,Mabaruma,GY,84.45,overcast clouds,8.2000,-59.7833,
9,Victoria,HK,66.25,overcast clouds,22.2855,114.1577,
10,Tuskegee,US,73.22,broken clouds,32.4240,-85.6916,
11,Marawi,PH,65.64,overcast clouds,7.9986,124.2928,
12,Ponta Do Sol,PT,65.03,overcast clouds,32.6667,-17.1000,
15,Anloga,GH,84.42,scattered clouds,5.7947,0.8973,
18,Marsh Harbour,BS,75.70,clear sky,26.5412,-77.0636,
19,Calama,CL,79.29,overcast clouds,-22.4667,-68.9333,


In [12]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
   
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [13]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Bredasdorp,ZA,65.52,light rain,-34.5322,20.0403,Bredasdorp Country Manor
6,Georgetown,MY,79.90,few clouds,5.4112,100.3354,Cititel Penang
8,Mabaruma,GY,84.45,overcast clouds,8.2000,-59.7833,Monty Brooms Guest House
9,Victoria,HK,66.25,overcast clouds,22.2855,114.1577,Mini Hotel Central
10,Tuskegee,US,73.22,broken clouds,32.4240,-85.6916,Macon Motel
...,...,...,...,...,...,...,...
2021,Bhigvan,IN,72.03,clear sky,18.3167,74.7667,Hotel Sagar
2022,Inyonga,TZ,74.25,overcast clouds,-6.7167,32.0667,Eden Rock Lodge
2023,Mayor Pablo Lagerenza,PY,93.85,light rain,-19.9309,-60.7718,
2024,Turiacu,BR,83.61,broken clouds,-1.6633,-45.3717,Pousada Maranata


In [14]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [15]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [16]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# 11b. Display the figure
fig = gmaps.figure(center = (37.0, -80.0), zoom_level = 1.5)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))